In [24]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [25]:
#Read the pdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")
documents=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='***Süd-Balkon - 3 Zimmer - Ihre\nneue attraktive Wohnung in\nGroßhadern***\nObjekt: 9664_02 • Terofalstr. 6 • 80689 München\n450.000,00 e\nMünchener Kindl Immobilien GmbH • Planegger Straße 21 • 82152 Martinsried\nTel.: +49 89 / 550 550 • Fax: +49 89 / 550 555-55\nI.Kornilenko@550550.de • http://www.muenchener-kindl.de', metadata={'source': 'us_census\\DEU_9664_02_lang_expose.pdf', 'page': 0})

In [26]:
##Embedding using hugginface 
huggingface_embeddings=HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
    
)


In [27]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-1.13845207e-02  3.14493030e-02 -2.21738149e-03  3.81870680e-02
  6.44704923e-02  7.72809051e-03  6.04445348e-03  3.39441337e-02
 -8.18114132e-02 -9.93905868e-03  2.57727765e-02 -9.68551859e-02
 -3.35184694e-03  3.88856046e-02  1.15057081e-02  2.93979794e-03
  3.01670302e-02  7.21270405e-03 -3.13902497e-02 -2.15509962e-02
  5.48738576e-02 -4.81020100e-02  6.85518011e-02  6.82288408e-03
  2.95396149e-02  5.32392375e-02 -6.37926743e-04 -5.37849916e-03
 -6.75855950e-02 -1.95969135e-01 -7.33470311e-03 -1.24461148e-02
 -1.91177931e-02 -2.45999210e-02  1.55035695e-02 -1.13167167e-02
 -3.07555217e-02  7.31115276e-03  3.21242101e-02  4.58281487e-02
  3.32756750e-02  2.96213999e-02 -2.29268279e-02  1.38131017e-02
  3.46129760e-02 -2.29880530e-02  1.78774502e-02  6.83086878e-03
 -1.64868142e-02 -1.35579854e-02 -8.80819038e-02 -2.94984062e-03
  4.05618437e-02 -4.53052996e-03 -5.90961333e-03  9.54034086e-03
  1.07420785e-02 -5.25949290e-03 -3.71073112e-02 -2.46021431e-02
  3.16405408e-02 -1.06736

In [28]:
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [55]:
## Query using Similarity Search
query="who is contact person?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

Flur Abstellkammer
Schlafzimmer Schlafzimmer
Schätz Immobilien GmbH
Maria-Merian-Str . 8 T el.: 089/954283-205 elke.schaetz@schaetz-immobilien.de
85521 Ottobrunn Fax: 089/954283-209 www.schaetz-immobilien.de


In [30]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001CCD048D990> search_kwargs={'k': 3}


In [57]:
import os
from dotenv import load_dotenv
load_dotenv()


os.environ['HUGGINGFACEHUB_API_TOKEN']="hf_UHwCPQliyRQNtWvrUmkvnGZTohqXxiQprwD521"

In [59]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the apartment address?"
hf.invoke(query)


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: vfmDhlDbdrlW2UBz5bLLP)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate